In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn import tree, preprocessing
import sklearn.ensemble as ske
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

# Input data files are available in the "../input/" directory.

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

188-million-us-wildfires
train-data



In [2]:
fires = pd.read_csv('/kaggle/input/train-data/train.csv')

In [3]:
fires.head()
fires.drop_duplicates() # removing the duplicate rows

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,FIPS_NAME,Shape,FIRE_SIZE_SCALED
0,1623287,201658541,SFO-NY-2012-NY4414-2012-050012,NONFED,ST-NASF,ST/C&L,USNYNYX,Fire Department of New York,NY4414,Fire Department of New York,...,0.1,A,41.147000,-73.927000,14.0,MISSING/NOT SPECIFIED,NY,Rockland,b'\x00\x01\xad\x10\x00\x00\x14\xd9\xce\xf7S{R\...,1.647431e-07
1,1569451,201435787,FS-1509063,FED,FS-FIRESTAT,FS,USORUPF,Umpqua National Forest,0615,Umpqua National Forest,...,0.1,A,43.084444,-122.617778,5.0,USFS,OR,Douglas,b'\x00\x01\xad\x10\x00\x00\xe8R\xd0\xab\x89\xa...,1.647431e-07
2,329243,336663,W-522697,FED,DOI-WFMI,BLM,USIDTFD,Twin Falls District,IDJRD,Jarbidge Field Office,...,98.6,C,42.741900,-114.848900,1.0,BLM,ID,Twin Falls,b'\x00\x01\xad\x10\x00\x00\xc0d\xaa`T\xb6\\\xc...,1.624529e-04
3,1218664,1476284,W-603889,FED,DOI-WFMI,BLM,USUTSLD,Salt Lake Field Office,UTSLD,Salt Lake Field Office,...,0.5,B,40.293300,-111.977500,8.0,PRIVATE,UT,Utah,b'\x00\x01\xad\x10\x00\x00\xc0\xf5(\\\x8f\xfe[...,8.237814e-07
4,1749209,300090002,SFO-2014GAGAS-FY2014-Wilcox-004,NONFED,ST-NASF,ST/C&L,USGAGAS,Georgia Forestry Commission,GAGAS,Georgia Forestry Commission,...,11.8,C,31.949458,-83.289372,8.0,PRIVATE,GA,Wilcox,b'\x00\x01\xad\x10\x00\x00TQ\x0f\x13\x85\xd2T\...,1.944161e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713600,261367,266414,W-536509,FED,DOI-WFMI,BIA,USSDCRA,Cheyenne River Agency,SDCRA,Cheyenne River Agency,...,1.8,B,45.202500,-101.497700,9.0,TRIBAL,SD,Unknown,b'\x00\x01\xad\x10\x00\x00\x04\xce\x19Q\xda_Y\...,2.965656e-06
713601,372213,384326,FWS-2007MSMSRC9G9,FED,FWS-FMIS,FWS,USMSMSR,Mississippi Sandhill Crane National Wildlife R...,MSMSR,Mississippi Sandhill Crane National Wildlife R...,...,0.2,A,30.440000,-88.639444,4.0,FWS,MS,Unknown,"b'\x00\x01\xad\x10\x00\x00,Z_\xa8\xec(V\xc0\x8...",3.295027e-07
713602,132152,133461,FS-358879,FED,FS-FIRESTAT,FS,USIDSCF,Salmon-Challis National Forest,0413,Salmon-Challis National Forest,...,0.2,A,44.906667,-114.291111,5.0,USFS,ID,Unknown,b'\x00\x01\xad\x10\x00\x00\xf0<~\x90\xa1\x92\\...,3.295027e-07
713603,1498456,201171855,2011SCSCS12FF0002,NONFED,ST-NASF,ST/C&L,USSCSCS,South Carolina Forestry Commission,SCSCS,South Carolina Forestry Commission,...,0.3,B,33.893082,-79.788811,8.0,PRIVATE,SC,Florence,b'\x00\x01\xad\x10\x00\x00d\xee!\xe1{\xf2S\xc0...,4.942622e-07


In [4]:
fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 713605 entries, 0 to 713604
Data columns (total 30 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   OBJECTID                    713605 non-null  int64  
 1   FOD_ID                      713605 non-null  int64  
 2   FPA_ID                      713605 non-null  object 
 3   SOURCE_SYSTEM_TYPE          713605 non-null  object 
 4   SOURCE_SYSTEM               713605 non-null  object 
 5   NWCG_REPORTING_AGENCY       713605 non-null  object 
 6   NWCG_REPORTING_UNIT_ID      713605 non-null  object 
 7   NWCG_REPORTING_UNIT_NAME    713605 non-null  object 
 8   SOURCE_REPORTING_UNIT       713605 non-null  object 
 9   SOURCE_REPORTING_UNIT_NAME  713605 non-null  object 
 10  FIRE_NAME                   711357 non-null  object 
 11  FIRE_YEAR                   713605 non-null  int64  
 12  DISCOVERY_DATE              713605 non-null  float64
 13  DISCOVERY_DOY 

In [5]:
fires.columns

Index(['OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM',
       'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
       'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
       'SOURCE_REPORTING_UNIT_NAME', 'FIRE_NAME', 'FIRE_YEAR',
       'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME', 'STAT_CAUSE_CODE',
       'STAT_CAUSE_DESCR', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE',
       'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR',
       'STATE', 'FIPS_NAME', 'Shape', 'FIRE_SIZE_SCALED'],
      dtype='object')

#### RandomForestRegressor

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


# Select features and target variable
features = ["FIRE_YEAR", "STAT_CAUSE_DESCR", "LATITUDE", "LONGITUDE", "STATE", "DISCOVERY_DATE"]
X = fires[features]
y = fires["FIRE_SIZE"]

# Convert categorical variables to one-hot encoding
X = pd.get_dummies(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Regressor
rf = RandomForestRegressor(n_estimators=50, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model
y_pred_train = rf.predict(X_train)
y_pred_test = rf.predict(X_test)
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
print("MSE on training set:", mse_train)
print("MSE on test set:", mse_test)

MSE on training set: 2105034.0596975796
MSE on test set: 14030071.403111005


#### RandomForestClassifier to predict FIRE_SIZE

In [8]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Select features and target variable
features = ["FIRE_YEAR", "STAT_CAUSE_DESCR", "LATITUDE", "LONGITUDE", "STATE", "DISCOVERY_DATE", "FIRE_SIZE"]
X = fires[features]
y = fires["FIRE_SIZE_CLASS"]

# Convert categorical variables to one-hot encoding
X = pd.get_dummies(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
rf = RandomForestClassifier(n_estimators=50, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model
y_pred_train = rf.predict(X_train)
y_pred_test = rf.predict(X_test)
acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)
print("Accuracy on training set:", acc_train)
print("Accuracy on test set:", acc_test)

Accuracy on training set: 1.0
Accuracy on test set: 0.9947940387188992
